<a href="https://colab.research.google.com/github/JF11579/Plotly_RE/blob/main/HomePrices_and_Home_Ins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Treat this like a HW assignment.

Home ins prices have gone up a lot.  Is that effecting the price of homes.

Start with multi line chart showing:

** household median incom, avg
    Units:  2021 CPI-U-RS Adjusted Dollars, Not Seasonally Adjusted

    https://fred.stlouisfed.org/series/MEHOINUSA672N

    Title:  Real Median Household Income in the United States (MEHOINUSA672N)


** avg homeowners ins.
    PPI industry data for Direct property and casualty insurers-Homeowner's insurance, not seasonally adjusted

    Base data 1998

    https://data.bls.gov/pdq/SurveyOutputServlet


** median home price
  https://fred.stlouisfed.org/series/MSPUS
  no price adjustment.  Nominal prices instead, current at that time.  do we apply CPI?

 **  HomeOwnership Rate

    Title: Homeownership Rate for the United States (USHOWN)

    https://fred.stlouisfed.org/series/USHOWN

Adjust data to common base year

The WSJ article [Americans Are Bailing on Their Home Insurance](https://www.wsj.com/personal-finance/americans-are-bailing-on-their-home-insurance-e3395515?page=1) discusses the trend of Americans giving up on their homeowner's insurance as the cost prices steeply.

# Notes

# Libraries

In [ ]:
# pip install dash plotly


In [8]:
import pandas as pd
from google.colab import drive
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go

# Data

In [72]:


drive.mount('/content/drive')
# Change to the directory containing your CSV
%cd /content/drive/MyDrive/DATA/HomeOwnershipData
file_path = 'Home_Ins_Cost.txt'
Home_Insurance_Cost= pd.read_csv(file_path ,delimiter=',')
Home_Insurance_Cost.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/DATA/HomeOwnershipData


,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Annual
0,1998,,,,,,100.0,100.1,100.4,100.6,101.1,101.2,101.5,
1,1999,101.9,101.9,102.2,102.3,102.7,102.9,103.3,103.5,103.7,104.1,104.2,104.3,103.1
2,2000,104.6,104.9,105.0,105.5,106.0,106.7,107.1,107.4,108.0,108.1,108.2,108.5,106.7
3,2001,109.4,110.4,110.7,110.7,111.3,111.5,112.0,112.3,112.6,113.3,113.8,114.0,111.8
4,2002,114.5,114.8,115.5,116.0,116.1,117.1,118.4,118.9,119.9,122.0,123.6,124.0,118.4


In [73]:
HomeOwnershipRates = pd.read_csv('HomeownershipRates.csv')
HomeOwnershipRates.head(3)

,DATE,USHOWN
0,1984-01-01,64.5
1,1985-01-01,63.9
2,1986-01-01,63.8


In [74]:
HouseholdMedianIncome = pd.read_csv('HouseholdMedianIncome.csv')
HouseholdMedianIncome.head(3)

,DATE,MEHOINUSA672N
0,2008-01-01,63455
1,2009-01-01,63011
2,2010-01-01,61364


In [75]:
US_MedianHomePrice = pd.read_csv('US_MedianHomePrice.csv')
US_MedianHomePrice.head(3)

,DATE,MSPUS
0,1963-01-01,17800
1,1963-04-01,18000
2,1963-07-01,17900


## Data Clean-Up

* Drop the month and date

* get all in same data range

* figure out the different scales



In [ ]:
#HomeOwnershipRates
#HomeOwnershipRates.head(3)

In [76]:
# prompt: I ahve a dolumn called Date with data like this 1984-01-01 and am only intersted in the year.  How do I drop the month and day from the column?

HomeOwnershipRates['DATE'] = HomeOwnershipRates['DATE'].map(lambda x: x[:4])
# HouseholdMedianIncome
HouseholdMedianIncome['DATE'] = HouseholdMedianIncome['DATE'].map(lambda x: x[:4])
US_MedianHomePrice['DATE'] = US_MedianHomePrice['DATE'].map(lambda x: x[:4])

In [ ]:
#HomeownershipRates.head(3)


In [54]:
# HouseholdMedianIncome
#HouseholdMedianIncome['DATE'] = HouseholdMedianIncome['DATE'].map(lambda x: x[:4])

In [ ]:
#HouseholdMedianIncome.head()

In [55]:
# US_MedianHomePrice
#US_MedianHomePrice['DATE'] = US_MedianHomePrice['DATE'].map(lambda x: x[:4])

In [ ]:
#US_MedianHomePrice.head(3)

## Lets rename all of the DATE cols to YEAR

In [77]:
# prompt: how do i rename the DATE column in HomeownershipRates to year? Home_Insurance_Cost
Home_Insurance_Cost.rename(columns={'Year': 'YEAR'}, inplace=True)
HomeOwnershipRates.rename(columns={'DATE': 'YEAR'}, inplace=True)
HouseholdMedianIncome.rename(columns={'DATE': 'YEAR'}, inplace=True)
US_MedianHomePrice.rename(columns={'DATE': 'YEAR'}, inplace=True)


## Our  household Medain Income Dataset starts only in 2008 so we willl limit the other datasets to this as well

## First convert all YEAR columns to integers

In [78]:
# prompt: in the HomeownershipRates dataframe how do I convert the YEAR column to integer

Home_Insurance_Cost['YEAR'] = Home_Insurance_Cost['YEAR'].astype(int)
HomeOwnershipRates['YEAR'] = HomeOwnershipRates['YEAR'].astype(int)
HouseholdMedianIncome['YEAR'] = HouseholdMedianIncome['YEAR'].astype(int)
US_MedianHomePrice['YEAR'] = US_MedianHomePrice['YEAR'].astype(int)

In [79]:
# prompt: how do i limit date for HomeownershipRates starting in the Year 2008?

Home_Insurance_Cost = Home_Insurance_Cost[Home_Insurance_Cost['YEAR'] >= 2008]
#HomeOwnershipRates = HomeOwnershipRates[HomeOwnershipRates['YEAR'] >= 2008]
US_MedianHomePrice = US_MedianHomePrice[US_MedianHomePrice['YEAR'] >= 2008]

# HomeownershipRates.head(3)

# Basement

In [3]:
app = dash.Dash(__name__)

# Sample data
years = [2010, 2011, 2012, 2013, 2014, 2015]
data_set1 = [100, 150, 200, 180, 250, 300]
data_set2 = [80, 120, 160, 140, 190, 220]
data_set3 = [120, 100, 150, 130, 180, 210]

app.layout = html.Div([
    dcc.Graph(id='multi-line-graph'),
])

@app.callback(
    Output('multi-line-graph', 'figure'),
    Input('multi-line-graph', 'hoverData')
)
def update_graph(hoverData):
    trace1 = go.Scatter(x=years, y=data_set1, mode='lines', name='Data Set 1')
    trace2 = go.Scatter(x=years, y=data_set2, mode='lines', name='Data Set 2')
    trace3 = go.Scatter(x=years, y=data_set3, mode='lines', name='Data Set 3')

    layout = go.Layout(
        title='Multi-Line Graph',
        xaxis={'title': 'Years'},
        yaxis={'title': 'Price'},
        hovermode='closest'
    )

    return {'data': [trace1, trace2, trace3], 'layout': layout}

if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>